In [ ]:
import os
import kagglehub
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
path = kagglehub.dataset_download("iamsouravbanerjee/animal-image-dataset-90-different-animals")
print("Dataset Path:", path)

train_dir = os.path.join(path, "animals")

print("Folders inside animals:", len(os.listdir(train_dir)))

In [ ]:
Trian_Datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
batch_size= 32
img_height= 150
img_width= 150

In [ ]:
Train_Genrator = Trian_Datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

Found 4320 images belonging to 1 classes.


In [ ]:
Val_Genrator = Trian_Datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

In [ ]:
print("Number of classes:", Train_Genrator.num_classes)
print("Class indices:", Train_Genrator.class_indices)

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(512, activation='relu'),
    Dense(Train_Genrator.num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
history = model.fit(
    Train_Genrator,
    steps_per_epoch=Train_Genrator.samples // batch_size,
    validation_data=Val_Genrator,
    validation_steps=Val_Genrator.samples // batch_size,
    epochs=10,
    verbose=1
)

In [ ]:
import os

for root, dirs, files in os.walk("/kaggle/input"):
    for file in files[:5]:
        print(os.path.join(root, file))

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

In [ ]:
img_path = "/kaggle/input/animal-image-dataset-90-different-animals/animals/animals/rhinoceros/35ec5e7a7d.jpg"

In [ ]:
img = image.load_img(img_path, target_size=(150,150))

In [ ]:
plt.imshow(img)
plt.axis("off")

In [ ]:
img_array = image.img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)

In [ ]:
pred = model.predict(img_array)

In [ ]:
class_labels = list(Train_Genrator.class_indices.keys())
pred_index = np.argmax(pred)
pred_lable = class_labels[pred_index]

In [ ]:
confidence = np.max(pred) * 100

In [ ]:
plt.title(f'predicted: {pred_lable} ({confidence:.2f}%)')
plt.show()

In [ ]:
print("Predicted Animal", pred_lable)
print("confidence:", confidence)